In [1]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 2  # Number of examples to generate for each emotion

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.6,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates realistic and stylistically diverse English tweets for the specified emotion labels. Each tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet for the emotion: {emotion}. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets.csv', index=False)
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

In [ ]:
# The code below illustrates how the prompt is constructed:
# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# Loop through each emotion label and print the prompts
for emotion in emotions:
    for _ in range(1):  # Generate 1 example for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
Generate a tweet for the emotion: {emotion}. There must only be one label tagged as 'True'. The tweet should be stylistically similar to the example below:
{example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        print(prompt)  # Print the prompt to check if it is correct
        print()  # Print a blank line for better readability

Given the following label: anger (also includes annoyance, rage)
Generate a tweet for the emotion: anger. There must only be one label tagged as 'True'. The tweet should be stylistically similar to the example below:
I have a rage rage ep 2 coming out soon I'll keep you posted on it #YouTube #youtubegaming #rage

Given the following label: anticipation (also includes interest, vigilance)
Generate a tweet for the emotion: anticipation. There must only be one label tagged as 'True'. The tweet should be stylistically similar to the example below:
should I #start posting #photos?

Given the following label: disgust (also includes disinterest, dislike, loathing)
Generate a tweet for the emotion: disgust. There must only be one label tagged as 'True'. The tweet should be stylistically similar to the example below:
@Montel_Williams If this ban goes through, it will harm so many people, disabled veterans, people who have #chronicpain, #iamkratom

Given the following label: fear (also includes 

Generating 100 examples for each label.

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet for the emotion: {emotion}. Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_no_role.csv', index=False)
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

First personality.

Neuroticism: MEDIUM

Extraversion: LOW

Oppenness: HIGH

Agreeableness: LOW

Conscientiousness: HIGH

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# System prompt
system_prompt = """You are a person who received the following results on the Big Five Personality test:
Neuroticism: MEDIUM
Extraversion: LOW
Oppenness: HIGH
Agreeableness: LOW
Conscientiousness: HIGH"""

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet in accordance with your personality score for the emotion: {emotion}. Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_role_1.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


Second personality.

Neuroticism: LOW

Extraversion: MEDIUM

Oppenness: MEDIUM

Agreeableness: HIGH

Conscientiousness: LOW

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# System prompt
system_prompt = """You are a person who received the following results on the Big Five Personality test:
Neuroticism: LOW
Extraversion: MEDIUM
Oppenness: MEDIUM
Agreeableness: HIGH
Conscientiousness: LOW"""

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet in accordance with your personality score for the emotion: {emotion}. Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_role_2.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


Third personality.

Neuroticism: HIGH

Extraversion: HIGH

Oppenness: HIGH

Agreeableness: HIGH

Conscientiousness: HIGH

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# System prompt
system_prompt = """You are a person who received the following results on the Big Five Personality test:
Neuroticism: HIGH
Extraversion: HIGH
Oppenness: HIGH
Agreeableness: HIGH
Conscientiousness: HIGH"""

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet in accordance with your personality score for the emotion: {emotion}. Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_role_3.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


Fourth personality.

Neuroticism: LOW

Extraversion: LOW

Oppenness: LOW

Agreeableness: LOW

Conscientiousness: LOW

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# System prompt
system_prompt = """You are a person who received the following results on the Big Five Personality test:
Neuroticism: LOW
Extraversion: LOW
Oppenness: LOW
Agreeableness: LOW
Conscientiousness: LOW"""

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet in accordance with your personality score for the emotion: {emotion}. Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_role_4.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


Fifth personality + news topics...?

Neuroticism: MEDIUM

Extraversion: LOW

Oppenness: HIGH

Agreeableness: LOW

Conscientiousness: HIGH

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# System prompt
system_prompt = """You are a person who received the following results on the Big Five Personality test:
Neuroticism: MEDIUM
Extraversion: LOW
Openness: HIGH
Agreeableness: LOW
Conscientiousness: HIGH"""

news_articles = ["Bangladesh parliament dissolved after PM Sheikh Hasina shocking exit",
                 "Global markets steady after sharp price falls",
                 "US funeral home fined $950m in decaying bodies case",
                 "Biden meets national security team as fears of Iran attack on Israel grow",
                 "Google's online search monopoly is illegal, US judge rules",
                 "Kamala Harris poised to announce her running mate",
                 "Five dead as Tropical Storm Debby soaks south-eastern US",
                 "BMW set upon and Asian men inside attacked - how violence surged in one UK city",
                 "Protests reveal deep-rooted anger, but UK is not at boiling point",
                 "Angered by Paris ban, Russia's media scorns 'the Olympics of Hell'",
                 "Is the US really heading for recession?",
                 "Somali police seize hundreds of veils amid security fears",
                 "Flintoff reveals 'nightmares' of Top Gear crash",
                 "Russian dissident tells BBC he thought he would die in 'Putin's prison'",
                 "Mateta leads France past Egypt and into Olympic final",
                 "Atletico agree £81.5m deal for Man City's Alvarez",
                 "Dead bear another strange twist in RFK Jr's faltering campaign",
                 "Man nearly hit when small plane crashes on golf course",
                 "The provocative 80s rap that became an anthem",
                 "'Cold Hawaii': Denmark's unlikely surf town"]

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Variable to track the current news article index
current_article_index = 0

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        # Get the current news article
        current_article = news_articles[current_article_index]

        # Update the prompt to include the news article headline
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet in accordance with your personality score for the emotion: {emotion}. The tweet should be on the topic of this news headline: {current_article} Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}."""

        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

        # Move to the next news article, looping back to the start if necessary
        current_article_index = (current_article_index + 1) % len(news_articles)

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_role_5.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found


CSV file generated successfully!


GPT + news no role

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

news_articles = ["Bangladesh parliament dissolved after PM Sheikh Hasina shocking exit",
                 "Global markets steady after sharp price falls",
                 "US funeral home fined $950m in decaying bodies case",
                 "Biden meets national security team as fears of Iran attack on Israel grow",
                 "Google's online search monopoly is illegal, US judge rules",
                 "Kamala Harris poised to announce her running mate",
                 "Five dead as Tropical Storm Debby soaks south-eastern US",
                 "BMW set upon and Asian men inside attacked - how violence surged in one UK city",
                 "Protests reveal deep-rooted anger, but UK is not at boiling point",
                 "Angered by Paris ban, Russia's media scorns 'the Olympics of Hell'",
                 "Is the US really heading for recession?",
                 "Somali police seize hundreds of veils amid security fears",
                 "Flintoff reveals 'nightmares' of Top Gear crash",
                 "Russian dissident tells BBC he thought he would die in 'Putin's prison'",
                 "Mateta leads France past Egypt and into Olympic final",
                 "Atletico agree £81.5m deal for Man City's Alvarez",
                 "Dead bear another strange twist in RFK Jr's faltering campaign",
                 "Man nearly hit when small plane crashes on golf course",
                 "The provocative 80s rap that became an anthem",
                 "'Cold Hawaii': Denmark's unlikely surf town"]

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Variable to track the current news article index
current_article_index = 0

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        # Get the current news article
        current_article = news_articles[current_article_index]

        # Update the prompt to include the news article headline
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet for the emotion: {emotion}. The tweet should be on the topic of this news headline: {current_article}. Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}."""

        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

        # Move to the next news article, looping back to the start if necessary
        current_article_index = (current_article_index + 1) % len(news_articles)

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_no_role_news.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


Generating 100 examples of single-label tweets.

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for the emotion label. The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion

        # Update the prompt
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet for the emotion: {emotion}. Ensure that only the label for {emotion} is True. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}."""

        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_single_label_no_role.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


Generating 100 examples of single-label tweets with a role.

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# System prompt
system_prompt = """You are a person who received the following results on the Big Five Personality test:
Neuroticism: MEDIUM
Extraversion: LOW
Openness: HIGH
Agreeableness: LOW
Conscientiousness: HIGH"""

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for the emotion label. The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion

        # Update the prompt
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet in accordance with your personality score for the emotion: {emotion}. Ensure that only the label for {emotion} is True. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}."""

        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_single_label_role_1.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


Generating 100 examples of single-label tweets with a role and news articles.

In [ ]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 100  # Number of examples to generate for each emotion

# System prompt
system_prompt = """You are a person who received the following results on the Big Five Personality test:
Neuroticism: MEDIUM
Extraversion: LOW
Openness: HIGH
Agreeableness: LOW
Conscientiousness: HIGH"""

news_articles = ["Bangladesh parliament dissolved after PM Sheikh Hasina shocking exit",
                 "Global markets steady after sharp price falls",
                 "US funeral home fined $950m in decaying bodies case",
                 "Biden meets national security team as fears of Iran attack on Israel grow",
                 "Google's online search monopoly is illegal, US judge rules",
                 "Kamala Harris poised to announce her running mate",
                 "Five dead as Tropical Storm Debby soaks south-eastern US",
                 "BMW set upon and Asian men inside attacked - how violence surged in one UK city",
                 "Protests reveal deep-rooted anger, but UK is not at boiling point",
                 "Angered by Paris ban, Russia's media scorns 'the Olympics of Hell'",
                 "Is the US really heading for recession?",
                 "Somali police seize hundreds of veils amid security fears",
                 "Flintoff reveals 'nightmares' of Top Gear crash",
                 "Russian dissident tells BBC he thought he would die in 'Putin's prison'",
                 "Mateta leads France past Egypt and into Olympic final",
                 "Atletico agree £81.5m deal for Man City's Alvarez",
                 "Dead bear another strange twist in RFK Jr's faltering campaign",
                 "Man nearly hit when small plane crashes on golf course",
                 "The provocative 80s rap that became an anthem",
                 "'Cold Hawaii': Denmark's unlikely surf town"]

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for the emotion label. The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Variable to track the current news article index
current_article_index = 0

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        # Get the current news article
        current_article = news_articles[current_article_index]

        # Update the prompt to include the news article headline
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet in accordance with your personality score for the emotion: {emotion}. The tweet should be on the topic of this news headline: {current_article}. Ensure that only the label for {emotion} is True. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}."""

        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

        # Move to the next news article, looping back to the start if necessary
        current_article_index = (current_article_index + 1) % len(news_articles)

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_single_label_role_1_news.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!


## Increased dataset

In [2]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 200  # Number of examples to generate for each emotion

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for each emotion label (anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust). The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet for the emotion: {emotion}. Ensure that other appropriate emotions (anger, anticipation, disgust, fear, joy, love, optimism, sadness, surprise, trust) are also present in the tweet that you generate. The {emotion} label must be tagged as 'True'. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}"""  # Create the prompt. Only relevant emotion label's explanation and example are used
        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_increased_multi.csv', index=False)
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

In [3]:
import openai  # Import the OpenAI library
import pandas as pd  # Import the pandas library for data handling

# Initialize the OpenAI client
client = openai.OpenAI(api_key='sk-proj-J3ORXe39g5rhHD7X8zoZT3BlbkFJHt16RKYRIE0ZYTvvW1HK')

n_examples = 200  # Number of examples to generate for each emotion

# Function to call the OpenAI model with a given prompt and function
def call_openai_model(prompt, function):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="gpt-4o-mini",  # The model to use
        functions=[function],
        function_call={"name": "generate_tweets"},
        temperature=0.7,  # The temperature for response variability
        max_tokens=200,  # The maximum number of tokens in the response
        top_p=1,  # The top-p value for nucleus sampling
        frequency_penalty=1,  # Frequency penalty to reduce repetition
        presence_penalty=1  # Presence penalty to encourage new topics
    )
    return response  # Return the response from the API

# Define the function for generating tweets
function = {
    "name": "generate_tweets",
    "description": "Generates a realistic English tweet for the specified emotion label. The tweet should be approximately 16 tokens long and should be formatted as a dictionary with the tweet text and boolean indicators for the emotion label. The generated tweets should resemble the provided examples in terms of style and structure.",
    "parameters": {
        "type": "object",
        "properties": {
            "Tweet": {"type": "string"},
            "anger": {"type": "boolean"},
            "anticipation": {"type": "boolean"},
            "disgust": {"type": "boolean"},
            "fear": {"type": "boolean"},
            "joy": {"type": "boolean"},
            "love": {"type": "boolean"},
            "optimism": {"type": "boolean"},
            "pessimism": {"type": "boolean"},
            "sadness": {"type": "boolean"},
            "surprise": {"type": "boolean"},
            "trust": {"type": "boolean"}
        },
        "required": ["Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
        # Ensuring each generated tweet includes the tweet text and all emotion indicators
    }
}

# Part of the prompt describing emotion labels
emotion_labels = {
  "anger": "anger (also includes annoyance, rage)",
  "anticipation": "anticipation (also includes interest, vigilance)",
  "disgust": "disgust (also includes disinterest, dislike, loathing)",
  "fear": "fear (also includes apprehension, anxiety, terror)",
  "joy": "joy (also includes serenity, ecstasy)",
  "love": "love (also includes affection)",
  "optimism": "optimism (also includes hopefulness, confidence)",
  "pessimism": "pessimism (also includes cynicism, no confidence)",
  "sadness": "sadness (also includes pensiveness, grief)",
  "surprise": "surprise (also includes distraction, amazement)",
  "trust": "trust (also includes acceptance, liking, admiration)"
}

# Example tweets for each emotion
example_tweets = {
    "anger": "I need some to help with my anger",
    "anticipation": "Let's get drunk and tell each other things we're afraid to say when we're sober.",
    "disgust": "I don't like pineapple I only eat them on pizza, they lose the sting when they get cooked.",
    "fear": "I'm so over having anxiety",
    "joy": "Watch this amazing live.ly broadcast by @izzybuzy365 #lively #musically",
    "love": "@lizbon @anomalily @gerikkransky Sorry for the levity if at all inappropriate but this combined with my love of Charlotte reminds me rn...",
    "optimism": "The point of living, and being an optimist, is to be foolish enough to believe the best is yet to come' - Peter Ustinov #optimism #quote",
    "pessimism": "Actually I always idealize the end of my relationships .. Such a pessimist",
    "sadness": "What day is it #lost",
    "surprise": "@SimplyMayaMarie @STILLStanding_B 😂😂😂 y'all know I'm crazy its just shocking that's all",
    "trust": "@ThatBritishDude no okay you don't not need to be going out of your way to make sure you are pleasing everyone do what makes you happy"
}

# List of emotion labels to generate tweets for
emotions = list(example_tweets.keys())

# List to store the generated tweets
tweets_data = []

# Loop through each emotion label
for emotion in emotions:
    for _ in range(n_examples):  # Generate the desired number of examples for each emotion

        # Update the prompt
        prompt = f"""Given the following label: {emotion_labels[emotion]}
        Generate a tweet for the emotion: {emotion}. Ensure that only the label for {emotion} is True. The tweet should be stylistically similar to the example below:
        {example_tweets[emotion]}."""

        response = call_openai_model(prompt, function)  # Call the model with the prompt and function
        if response.choices:  # Check if there are choices in the response
            choice = response.choices[0]  # Access the first choice directly
            if choice.message and choice.message.function_call:  # Check if the choice has a message and function call
                function_call = choice.message.function_call
                if function_call.arguments:  # Check if the function call has arguments
                    tweets_data.append(function_call.arguments)  # Append the arguments to the tweets_data list

# Create a DataFrame from the collected tweets data if not empty
if tweets_data:
    df = pd.DataFrame(tweets_data)
    # Save the DataFrame to a CSV file
    df.to_csv('generated_tweets_increased_single.csv', index=False)
    print("CSV file generated successfully!")
else:
    print("No tweets data found in the response.")  # Print a message if no tweets data is found

CSV file generated successfully!
